In [3]:
import glob
import pandas as pd

In [6]:
def stda_to_csv(filename):
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        oscs = []
        energyEV = []
        while line:
            if 'excitation energies, transition moments and TDA amplitudes' in line:
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                while line != '\n':
                    line_list = line.split()
                    oscs.append(float(line_list[3]))
                    energyEV.append(float(line_list[1]))
                    line = file.readline()
            line = file.readline()  
        line = file.readline()

        # Creates full spectrum
        (spectraEV,  spectraNM, spectraIntensity) = spectra(energyEV, oscs)

        df = pd.DataFrame()
        df['eV'] = spectraEV
        df['nm'] = spectraNM
        df['intensity'] = spectraIntensity
        
        return df

def spectra(etens, etoscs, low = 0.5, high = 10.0, resolution = 0.01, smear = 0.04):
    """
    Return arrays of the energies and intensities of a Lorentzian-blurred spectrum

    Parameters
    ----------
    etens: list
        list of transition energies in units of eV
    etoscs: list
        list of oscillator strengths
    low: float
        transition in eV to start spectrum at
    high: float
        transition in eV to end spectrum at
    resolution: float
        increments of eV for spectrum
    smear: float
        blurs intensities of peaks across 0.04 eV

    Returns
    -------
    Lists of the spectra in eV, nm, and their oscillator strengths
    """
    maxSlices = int((high - low) / resolution) + 1
    peaks = len(etens)

    spectraEV = []
    spectraNM = []
    spectraIntensity = []
    for i in range(0, maxSlices):
        energy = float(i * resolution + low) # units of eV
        #wavelength = energy * 1239.84193 # convert eV to nm  
        wavelength = 1239.84193 / energy # convert eV to nm  
        intensity = 0.0

        for trans in range(0, peaks):
            this_smear = smear / 0.2 * (-0.046 * etoscs[trans] + 0.20)
            deltaE = etens[trans] - energy
            intensity = intensity + etoscs[trans] * this_smear**2 / (deltaE**2 + this_smear**2)

        spectraEV.append(energy)
        spectraNM.append(wavelength) 
        spectraIntensity.append(intensity)
        
    return spectraEV, spectraNM, spectraIntensity
        

In [15]:
acceptors = pd.read_csv('top_acceptors_GA1234.csv')
acceptors.iloc[0]

acceptor    39_126_39
best PCE     21.11091
Name: 0, dtype: object

In [7]:
for x in range(len(acceptors)):
    acc = acceptors.iloc[x][0]
    acc_stda = 'sTDDFT_xTB_output_GA123_acceptors/' + acc + '.stda'
    df_acc = stda_to_csv(acc_stda)
    acc_filename_abs = 'acc_abs_csvs/' + acc + '_abs.csv'
    df_acc.to_csv(acc_filename_abs)
    
donors = pd.read_csv('top_donors_GA1234.csv')
for x in range(len(donors)):
    don = donors.iloc[x][0]
    don_stda = '../GA_4/sTDDFTxTB_output/' + don + '.stda'
    df_don = stda_to_csv(don_stda)
    
    don_filename_abs = 'don_abs_csvs/' + don + '_abs.csv'
    df_don.to_csv(don_filename_abs)
    


## Find max absorption integral and overlap for normalization for fitness function

In [ ]:
def overlap(don_spectraIntensity, acceptor_name):
    
    overlapped_spectra_intensities = [don_spectraIntensity[i] * acc_spectraIntensity[i] for i in range(len(don_spectraIntensity))]
    
    area_altered_spectra = np.trapz(np.flip(overlapped_spectra_intensities), np.flip(acc_spectraNM), dx=0.1, axis=- 1)    
        
    return area_altered_spectra


def combined_absorption(acc_abs, don_abs):
    
    combined_spectra_intensities = [don_spectraIntensity[i] + acc_spectraIntensity[i] for i in range(len(don_spectraIntensity))]
    
    # Calculates the area under the curve using trapz rule for integration
    area_spectra = np.trapz(np.flip(spectraIntensity), np.flip(spectraNM), dx=0.1, axis=- 1)